In [187]:
import importlib
import random
import mrcfile
import csv
from pathlib import Path

In [188]:
import pokemino_generator as poke

In [189]:
importlib.reload(poke)

<module 'pokemino_generator' from '/Users/anna/OneDrive - Nexus365/RFI/Code/3DTetrisGen/pokemino_generator.py'>

In [190]:
import yaml

## Exp1_S_vs_L

### Create a training set

In [108]:
seeds = [1, 2, 3, 4, 5, 6]
n_bricks = [3, 7, 12, 20, 20, 40]

for seed, size in zip(seeds, n_bricks):
    for theta_x in poke.THETA_EACH_90:
        for theta_y in poke.THETA_EACH_90:
            for theta_z in poke.THETA_EACH_90:
                volume = poke.Volume ((10, 10, 10))
                pokemino = poke.Pokemino3D(seed, size, volume, -1)
                pokemino.rotate_the_block_1_axis(axis=0, theta=theta_z)
                pokemino.rotate_the_block_1_axis(axis=1, theta=theta_x)
                pokemino.rotate_the_block_1_axis(axis=2, theta=theta_y)
                volume.fit_pokemino(pokemino)
                volume.create_subpixel_resolution_array(5)
                volume.save_as_mrcfile(Path(f'Exp1_S_vs_L/{seed}_{size}_{theta_z}_{theta_x}_{theta_y}.mrc'))

volume.display_in_napari()

In [110]:
# Visualise a scrap from the training dataset
with mrcfile.open(Path('Exp1_S_vs_L/4_20_270_270_270.mrc')) as mrc:
    viewer = napari.view_image(mrc.data)

### Create a test set

In [111]:
seeds = [1, 2, 3, 4, 7, 6]
n_bricks = [3, 7, 12, 20, 20, 40]

max_size = 10
j, k, l = 2, 10, 10 # numbers of polyominos on a regular grid in each dimension

test_volume = poke.Volume((max_size * j, max_size * k, max_size * l))

for x in range (j):
    for y in range (k):
        for z in range (l):
            random.seed()
            no_pokemino=random.randint(0,5)
            pokemino = poke.Pokemino3D(seeds[no_pokemino], n_bricks[no_pokemino], test_volume, positioning = (int(max_size*(x+0.5)), int(max_size*(y+0.5)), int(max_size*(z+0.5))))
            pokemino.rotate_the_block_3_axes()

test_volume.fit_all_pokeminos()
test_volume.create_subpixel_resolution_array(5)
test_volume.save_as_mrcfile(Path(f'Exp1_S_vs_L/test.mrc'))
test_volume.display_in_napari()

In [112]:
with open(Path(f'Exp1_S_vs_L/test_description.mrc'), 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['seed', 'size', 'x', 'y', 'z'])
    for creature in test_volume.creatures:
        x, y, z = creature.positioning
        writer.writerow([creature.seed, creature.size, x, y, z])

## Exp2_S_vs_L_w_Noise

### Create a test set

In [113]:
seeds = [1, 2, 3, 4, 7, 6]
n_bricks = [3, 7, 12, 20, 20, 40]

n_pokeminos = 200
a_volume = 60 
test_volume = poke.Volume ((a_volume, a_volume, a_volume))

for x in range (n_pokeminos):
    random.seed()
    no_pokemino=random.randint(0,5)
    polyomino = poke.Pokemino3D(seeds[no_pokemino], n_bricks[no_pokemino], test_volume, positioning=(random.randint(0, test_volume.shape[0]-1), random.randint(0, test_volume.shape[1]-1), random.randint(0, test_volume.shape[2]-1)), density=x)
    polyomino.rotate_the_block_3_axes()

test_volume.check_for_pairwise_overlap()
test_volume.fit_all_pokeminos(fit_excluded_volume=True)
test_volume.add_extra_volume_to_remove_cuts()
test_volume.create_subpixel_resolution_array(5)
test_volume.save_as_mrcfile(Path(f'Exp2_S_vs_L_w_Noise/test.mrc'))

In [114]:
test_volume.display_in_napari()

In [115]:
with open(Path(f'Exp2_S_vs_L_w_Noise/test_description.mrc'), 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['seed', 'size', 'x', 'y', 'z'])
    for creature in test_volume.creatures:
        x, y, z = creature.positioning
        writer.writerow([creature.seed, creature.size, x, y, z])

### Create a training set

In [195]:
seeds = [1, 2, 3, 4, 5, 6]
n_bricks = [3, 7, 12, 20, 20, 40]
n_subpixels = 5

n_pokeminos = 500
a_volume = 80 
test_volume = poke.Volume ((a_volume, a_volume, a_volume))

for x in range (n_pokeminos):
    random.seed()
    no_pokemino=random.randint(0,5)
    polyomino = poke.Pokemino3D(seeds[no_pokemino], n_bricks[no_pokemino], test_volume, positioning=(random.randint(0, test_volume.shape[0]-1), random.randint(0, test_volume.shape[1]-1), random.randint(0, test_volume.shape[2]-1)))
    polyomino.rotate_the_block_3_axes()

test_volume.check_for_pairwise_overlap()
test_volume.fit_all_pokeminos()
test_volume.create_subpixel_resolution_array(n_subpixels)

In [196]:
for i, creature in enumerate(test_volume.creatures):
    x, y, z = creature.positioning
    if all([i in range(5, 75) for i in [x, y, z]]):
        mrc = mrcfile.new(Path(f'Exp2_S_vs_L_w_Noise/{creature.seed}_{creature.size}_{i}.mrc'), overwrite=True)
        scrap = test_volume.subpixel_resolution_array[(x-5)*n_subpixels:(x+5)*n_subpixels, (y-5)*n_subpixels:(y+5)*n_subpixels, (z-5)*n_subpixels:(z+5)*n_subpixels]
        mrc.set_data(scrap)
        mrc.close()

In [198]:
# Visualise a scrap from the training dataset
with mrcfile.open(Path('Exp2_S_vs_L_w_Noise/4_20_261.mrc')) as mrc:
    viewer = napari.view_image(mrc.data)

## Algorithm for more extended Pokeminos

In [193]:
volume = poke.Volume((100, 50, 50))
pokeminko1 = poke.Pokemino3D(3, 500, volume, positioning=(25, 25, 25), algorithm="extended")
pokeminko2 = poke.Pokemino3D(3, 500, volume, positioning=(75, 25, 25), algorithm="clumped")
print(pokeminko1.bricks)
print(pokeminko2.bricks)
volume.fit_all_pokeminos()
volume.display_in_napari()

[Brick([0, -2, -1]) Brick([1, -2, -1]) Brick([1, -2, 0])
 Brick([1, -3, -1]) Brick([0, -2, -2]) Brick([1, -2, 1]) Brick([2, -2, 1])
 Brick([0, -1, -2]) Brick([1, -2, 2]) Brick([0, -2, -1]) Brick([3, -2, 1])
 Brick([-1, -2, -1]) Brick([3, -1, 1]) Brick([1, -1, 1])
 Brick([-1, -2, 0]) Brick([0, -2, -3]) Brick([-1, -2, -2])
 Brick([0, -3, -3]) Brick([-2, -2, -1]) Brick([3, -1, 2])
 Brick([2, -1, 2]) Brick([0, -1, 1]) Brick([-1, -1, -1]) Brick([1, -2, 3])
 Brick([1, -1, -2]) Brick([4, -1, 1]) Brick([0, 0, -2]) Brick([4, 0, 1])
 Brick([3, -1, 3]) Brick([0, -3, -1]) Brick([2, -3, -1]) Brick([1, -3, 2])
 Brick([0, -4, -1]) Brick([2, -2, 0]) Brick([0, -4, -3])
 Brick([1, -1, -1]) Brick([-1, -3, -2]) Brick([0, 1, -2])
 Brick([2, -3, 2]) Brick([0, 2, -2]) Brick([0, -2, 3]) Brick([0, 1, -1])
 Brick([-1, -2, 3]) Brick([1, 0, -2]) Brick([1, -1, 3]) Brick([-1, 2, -2])
 Brick([0, 1, 0]) Brick([1, -4, 2]) Brick([-1, 0, -2]) Brick([-1, 0, -3])
 Brick([1, -5, 2]) Brick([-1, -3, -3]) Brick([0, 2, -3]) Br

### random.py

In [ ]:
mrcfile.validate(Path('rotated_polyominos/1_0_0_0.mrc'))

with mrcfile.open(Path('rotated_polyominos/5_0_0_0.mrc')) as mrc:
    viewer = napari.view_image(mrc.data)

## Create a 2D Polyomino and visualise all 90 degrees rotations

heptamino = Pokemino2D(4, 10)
fig = plt.figure()

for i in range(1, 5):
    rotation = THETA_EACH_90[i-1]
    ax = fig.add_subplot(2, 2, i)
    heptamino.rotate_the_block(rotation)
    heptamino.visualise_the_2d_block(10)

## Exploring mrcfile library & displaying mrc in napari

# Displaying a 2D mrcfile
with mrcfile.new('tmp2d.mrc', overwrite=True) as mrc:
    mrc.set_data(np.zeros((5, 5), dtype=np.int8))
    mrc.data[1:4,1:4] = 10
    mrc.data[2:3,2:3] = 30
    viewer = napari.view_image(mrc.data)
    

# Displaying a 3D mrcfile
with mrcfile.new('tmp3d.mrc', overwrite=True) as mrc:
    mrc.set_data(np.zeros((5, 5, 5), dtype=np.int8))
    mrc.data[1:4,1:4,1:4] = 10
    mrc.data[2:3,2:3,2:3] = 30
    viewer = napari.view_image(mrc.data)